## Automating Detection of Non-Functioning Readers

 

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import configparser
from psycopg2 import connect
import psycopg2.sql as pg
import pandas.io.sql as pandasql
import io


CONFIG = configparser.ConfigParser()
CONFIG.read(r'C:\Users\alouis2\Documents\Python Scripts\db.cfg')
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

In [2]:
string = ''' SELECT segments.analysis_id
               FROM bluetooth.segments
            EXCEPT
             SELECT DISTINCT f.analysis_id
               FROM (SELECT observations.analysis_id,
                        observations.measured_timestamp
                       FROM bluetooth.observations
                      WHERE observations.measured_timestamp <= (( SELECT max(observations_1.measured_timestamp) AS max
                               FROM bluetooth.observations observations_1)) AND observations.measured_timestamp >= (( SELECT max(observations_1.measured_timestamp) - '02:00:00'::interval
                               FROM bluetooth.observations observations_1)) AND (observations.analysis_id IN ( SELECT segments.analysis_id
                               FROM bluetooth.segments))
                      ORDER BY observations.measured_timestamp DESC) f'''

df = pandasql.read_sql(pg.SQL(string), con)


In [3]:
broken_readers = []
for i in range(len(df['analysis_id'])):
    string = '''SELECT analysis_id, startpoint_name, endpoint_name,  measured_timestamp as last_active FROM bluetooth.observations
                WHERE analysis_id = %d 
                ORDER BY measured_timestamp desc 
                LIMIT 1''' % list(df['analysis_id'])[i]
    row = pandasql.read_sql(pg.SQL(string), con)
    broken_readers.append(list(row.loc[0]))

broken_readers

[[1455710, 'QU_BV', 'EA_BV', Timestamp('2018-02-15 08:26:45')],
 [1455724, 'EA_BV', 'QU_BV', Timestamp('2018-02-15 08:13:30')],
 [1432978, 'BR3', 'BR4', Timestamp('2017-07-21 08:20:11')],
 [1454853, 'QU_RO', 'DU_RO', Timestamp('2017-12-17 17:57:21')],
 [1453680, 'QU_BV', 'QU_PA', Timestamp('2018-02-15 08:18:06')],
 [1454832, 'DU_RO', 'QU_RO', Timestamp('2017-12-17 18:44:11')],
 [1454311, 'QU_BV', 'KN_PA', Timestamp('2018-02-15 08:30:11')],
 [1453667, 'QU_PA', 'QU_BV', Timestamp('2018-02-15 08:01:21')],
 [1412684, 'AD2', 'AD1', Timestamp('2017-12-17 18:00:14')],
 [1412641, 'AD1', 'AD2', Timestamp('2017-12-17 18:47:53')],
 [1432982, 'BR2', 'BR3', Timestamp('2017-07-21 09:13:37')],
 [1453239, 'DU_RO', 'DU_DF', Timestamp('2017-12-17 18:51:17')],
 [1454293, 'KN_PA', 'QU_BV', Timestamp('2018-02-15 08:13:30')],
 [1453507, 'DU_DF', 'DU_RO', Timestamp('2017-12-17 17:59:21')]]

In [4]:
broken_readers = pd.DataFrame(broken_readers, columns = list(row.columns.values))

In [76]:
broken_readers = broken_readers.sort_values(by='last_active')
broken_readers

,analysis_id,startpoint_name,endpoint_name,last_active
2,1432978,BR3,BR4,2017-07-21 08:20:11
10,1432982,BR2,BR3,2017-07-21 09:13:37
3,1454853,QU_RO,DU_RO,2017-12-17 17:57:21
13,1453507,DU_DF,DU_RO,2017-12-17 17:59:21
8,1412684,AD2,AD1,2017-12-17 18:00:14
5,1454832,DU_RO,QU_RO,2017-12-17 18:44:11
9,1412641,AD1,AD2,2017-12-17 18:47:53
11,1453239,DU_RO,DU_DF,2017-12-17 18:51:17
7,1453667,QU_PA,QU_BV,2018-02-15 08:01:21
1,1455724,EA_BV,QU_BV,2018-02-15 08:13:30


In [21]:
string2 = '''SELECT analysis_id, startpoint_name, endpoint_name
            FROM bluetooth.observations
                WHERE observations.measured_timestamp <= (( SELECT max(observations_1.measured_timestamp) AS max
                FROM bluetooth.observations observations_1)) AND observations.measured_timestamp >= (( SELECT max(observations_1.measured_timestamp) - '02:00:00'::interval
                FROM bluetooth.observations observations_1)) AND (observations.analysis_id IN ( SELECT segments.analysis_id
                FROM bluetooth.segments))
                GROUP BY analysis_id, startpoint_name, endpoint_name;'''

df2 = pandasql.read_sql(pg.SQL(string2), con)


In [49]:
df2.head(5)

,analysis_id,startpoint_name,endpoint_name
0,116449,J,I
1,116451,I,J
2,116749,A,B
3,116796,F,G
4,116804,G,H


In [139]:
final = []
for i in range(len(broken_readers)): 
    if (broken_readers['startpoint_name'].values[i] not in df2['startpoint_name'].values \
    and broken_readers['startpoint_name'].values[i] not in df2['endpoint_name'].values):
        final.append(broken_readers['startpoint_name'].values[i])
for i in range(len(broken_readers)):
    if broken_readers['endpoint_name'].values[i] not in df2['startpoint_name'].values \
    and broken_readers['endpoint_name'].values[i] not in df2 ['endpoint_name'].values:
        final.append(broken_readers['endpoint_name'].values[i])
        
final = list(set(final))

for i in final: 
    j = (len(broken_readers))-1
    while j < len(broken_readers):
        if i == broken_readers['startpoint_name'].values[j]:
            final[final.index(i)] = ([i, broken_readers['last_active'].values[j]])
            break
        elif i == broken_readers['endpoint_name'].values[j]:
            final[final.index(i)] = ([i, broken_readers['last_active'].values[j]])
            break
        else:
            j = j-1

final = [i + [list(broken_readers.loc[broken_readers['startpoint_name'] == i[0]]['analysis_id'])+ \
         list(broken_readers.loc[broken_readers['endpoint_name'] == i[0]]['analysis_id'])] \
        for i in final]


In [142]:
broken = pd.DataFrame(final, columns = ['Reader', 'Last Active', 'Routes Affected'])
broken 

,Reader,Last Active,Routes Affected
0,QU_BV,2018-02-15 08:30:11,"[1453680, 1455710, 1454311, 1453667, 1455724, ..."
1,DU_RO,2017-12-17 18:51:17,"[1454832, 1453239, 1454853, 1453507]"
2,BR4,2017-07-21 08:20:11,[1432978]
3,BR3,2017-07-21 09:13:37,"[1432978, 1432982]"
4,AD1,2017-12-17 18:47:53,"[1412641, 1412684]"
